The goal of this notebook is to clean the Enron Email data set to perform email text summarization: Given a single person’s folder and a time range give a summary for each email. 

I plan to use a extractive summarization approach where key sentences are taken from the input email to form the summary. 

My approach to cleaning the data involves the following steps:

1. I have unpacked the maildir directory from the Enron data set. I will be focusing on each person’s inbox, which means all other folders will be ignored for now.

2. I will process each inbox email to add the following information to a Pandas data frame:

Employee: The username of the email owner. 
Body: Cleaned body of the email. 
Subject: The title of the email. 
From: The original sender of the email
Message-ID: Used to remove duplicate emails, as each email has a unique ID. 
Chain: The parsed out email chain from a email that was forwarded. 
Signature: The extracted signature from the body.
Date: Time the email was sent. 

3. During the email processing the full body of the email will be cleaned using a combination of delimiter splitting, and the talon email cleaning library to extract signatures. 

4. Null values will be set or cleaned up. 

5. Then the Date column will be set to Datetime. 

In [1]:
import mailparser
import sys
from os import listdir
from os.path import isfile, join
import pandas as pd
import email
import numpy as np
from talon.signature.bruteforce import extract_signature

First define the function that will clean up the email body. The following splits the chain emails from the actual body of the email. It appears that "----Original Message-----" indicates a forwarded part of an email so this is one of the delimiters. 

There are also emails that don't seem to necessarily be part of a chain, but would be referencing another email, which means more email metadata would be found in the body. If this meta data is found first, then the email will be split there instead. The goal is to cut out the largest amount of email forwarding and related metadata to extract only the written message by the original sender. 

Lastly Talon will be used to extract the signatures. 

In [2]:
def clean_body(mail_body):
    delimiters = ["-----Original Message-----","To:","From"]
    
    #Trying to split string by biggest delimiter. 
    old_len = sys.maxsize
    
    for delimiter in delimiters:
        split_body = mail_body.split(delimiter,1)
        new_len = len(split_body[0])
        if new_len <= old_len:
            old_len = new_len
            final_split = split_body
            
    #Then pull chain message
    if (len(final_split) == 1):
        mail_chain = None
    else:
        mail_chain = final_split[1] 
    
    #The following uses Talon to try to get a clean body, and seperate out the rest of the email. 
    clean_body, sig = extract_signature(final_split[0])
    
    return {'Body': clean_body, 'Chain' : mail_chain, 'Signature': sig}

The following function processes the email and appends the parsed pieces into a pandas dataframe. We need to start with a list, that will represent each row of the dataframe. Each item in the row is a dict of parsed email values.


In [3]:
enron_email_list = []
def process_email(email_path, employee, folder, email_name):
    mail = mailparser.parse_from_file(email_path)
    
    full_body = email.message_from_string(mail.body)
    mail_body = full_body.get_payload()    
    split_body = clean_body(mail_body)
    headers = mail.headers
    #Reformating date to be more pandas readable
    date_time = email.utils.format_datetime(email.utils.parsedate_to_datetime(headers.get('Date')))
    
    email_dict = {
                "Employee" : employee,
                "Email Folder": folder,
                "Message-ID": headers.get('Message-ID'),
                "Date" : date_time,
                "From" : headers.get('From'),
                "Subject": headers.get('Subject'),
                "Body" : split_body['Body'],
                "Chain" : split_body['Chain'],
                "Signature": split_body['Signature'],
                "Full_Email_Path" : email_path #for debug purposes. 
    }
    
    #Append row to dataframe. 
    enron_email_list.append(email_dict)

Now go through each person's inbox then load up each email to be cleaned and added to the dataframe. 

In [4]:
mail_dir = '../data/maildir/'
mailbox = "skilling-j" #testing
#mailboxes = listdir(mail_dir)

#for mailbox in mailboxes:
inbox = listdir(mail_dir + mailbox)
for folder in inbox:
    path = mail_dir + mailbox + "/" + folder
    emails = listdir(path)
    for single_email in emails:
        full_path = path + "/" + single_email
        process_email(full_path, mailbox, folder, single_email)
enron_email_list_df = pd.DataFrame(enron_email_list)
enron_email_list_df.head()

,Body,Chain,Date,Email Folder,Employee,From,Full_Email_Path,Message-ID,Signature,Subject
0,"Simon, I was unable to open the attached docum...",:\tSimone La rose/ENRON@enronXgate on 05/16/20...,"Wed, 16 May 2001 21:18:00 -0700",sent_items,skilling-j,jeff.skilling@enron.com,../data/maildir/skilling-j/sent_items/1.,<25701656.1075840162944.JavaMail.evans@thyme>,"Thanks,\nJoannie","Re: EnronOnline Executive Summary for May 15, ..."
1,"Erica,\n\nDue to the fact that Jeff is unable ...","\t""'jeff.skilling@enron.com'"" <jeff.skilling@e...","Mon, 14 May 2001 15:20:00 -0700",sent_items,skilling-j,jeff.skilling@enron.com,../data/maildir/skilling-j/sent_items/16.,<8541273.1075840163402.JavaMail.evans@thyme>,None,Re: Telephone Call with Jerry Murdock
2,"PLEASE PLAN TO ATTEND\n\n""NEW BUSINESS CR...",None,"Wed, 01 Aug 2001 12:25:58 -0700",sent_items,skilling-j,jeff.skilling@enron.com,../data/maildir/skilling-j/sent_items/110.,<20911721.1075852686924.JavaMail.evans@thyme>,None,Please Plan to Attend
3,"The Stanford Business School, in collaboration...",None,"Thu, 03 May 2001 18:30:00 -0700",sent_items,skilling-j,jeff.skilling@enron.com,../data/maildir/skilling-j/sent_items/28.,<20013668.1075840163679.JavaMail.evans@thyme>,None,Stanford/McKinsey GLOBE Survey
4,test,None,"Tue, 15 May 2001 14:09:00 -0700",sent_items,skilling-j,jeff.skilling@enron.com,../data/maildir/skilling-j/sent_items/5.,<30255134.1075840163074.JavaMail.evans@thyme>,None,test


Need to convert date to pandas datetime. 

In [5]:
enron_email_list_df['Date'] = pd.to_datetime(enron_email_list_df.Date)

In [6]:
enron_email_list_df.dtypes

Body                       object
Chain                      object
Date               datetime64[ns]
Email Folder               object
Employee                   object
From                       object
Full_Email_Path            object
Message-ID                 object
Signature                  object
Subject                    object
dtype: object

Describing the table shows that the Subject column has a large amount of blanks that should be set to null. All the other columns seem to make sense. 

In [7]:
enron_email_list_df.describe()

,Body,Chain,Date,Email Folder,Employee,From,Full_Email_Path,Message-ID,Signature,Subject
count,4139,1349,4139,4139,4139,4139,4139,4139,643,4139
unique,2386,684,2506,12,1,919,4139,4139,243,1960
top,The report named: Violation/Notification Memo ...,All Enron Worldwide\ncc: \nSubject: Successi...,2000-07-27 14:31:00,inbox,skilling-j,sherri.sera@enron.com,../data/maildir/skilling-j/discussion_threads/...,<16975694.1075840155315.JavaMail.evans@thyme>,"Regards,\nSherri Sera\nAssistant to Jeff Skill...",
freq,42,14,8,1252,4139,662,1,1,56,198
first,NaN,NaN,1980-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,2001-11-19 21:34:59,NaN,NaN,NaN,NaN,NaN,NaN,NaN


All the blanks will be replaced with NaN to keep values consistent.

In [8]:
test = enron_email_list_df.replace('',np.NaN)

In [9]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4139 entries, 0 to 4138
Data columns (total 10 columns):
Body               4115 non-null object
Chain              1348 non-null object
Date               4139 non-null datetime64[ns]
Email Folder       4139 non-null object
Employee           4139 non-null object
From               4139 non-null object
Full_Email_Path    4139 non-null object
Message-ID         4139 non-null object
Signature          643 non-null object
Subject            3941 non-null object
dtypes: datetime64[ns](1), object(9)
memory usage: 323.4+ KB


Taking a look at some examples.

In [15]:
enron_email_list_df.iloc[0]

Body               Simon, I was unable to open the attached docum...
Chain              :\tSimone La rose/ENRON@enronXgate on 05/16/20...
Date                                             2001-05-17 04:18:00
Email Folder                                              sent_items
Employee                                                  skilling-j
From                                         jeff.skilling@enron.com
Full_Email_Path             ../data/maildir/skilling-j/sent_items/1.
Message-ID             <25701656.1075840162944.JavaMail.evans@thyme>
Signature                                           Thanks,\nJoannie
Subject            Re: EnronOnline Executive Summary for May 15, ...
Name: 0, dtype: object

In [16]:
enron_email_list_df.iloc[0].Body

'Simon, I was unable to open the attached document for Jeff Skilling.  Could you please resend.'

In [17]:
enron_email_list_df.iloc[0].Signature


'Thanks,\nJoannie'

In [18]:
enron_email_list_df.iloc[0].Chain

':\tSimone La rose/ENRON@enronXgate on 05/16/2001 04:52 PM\nTo:\tSunil Abraham/ENRON@enronXgate, Cliff Baxter/ENRON@enronXgate, Sally Beck/ENRON@enronXgate, Tim Belden/ENRON@enronXgate, Philippe A Bibi/ENRON@enronXgate, Raymond Bowen/ENRON@enronXgate, Debbie R Brackett/ENRON@enronXgate, William S Bradford/ENRON@enronXgate, Michael Bridges/ENRON@enronXgate, Michael R Brown/LON/ECT@ECT, Lydia Cannon/ENRON@enronXgate, Bob Crane/ENRON@enronXgate, John Cummings/ENRON@enronXgate, Bryan Deluca/ENRON@enronXgate, Karen Denne/Corp/Enron@ENRON, Jennifer deBoisblanc Denny/ENRON@enronXgate, Daniel Diamond/ENRON@enronXgate, Marc Eichmann/ENRON@enronXgate, Jim Fallon/Enron Communications@Enron Communications, Jay Fitzgerald/NY/ECT@ECT, David Forster/ENRON@enronXgate, Mark Frevert/ENRON@enronXgate, Bruce Garner/LON/ECT@ECT, Paul Goddard/EU/Enron@Enron, Joe Gold/LON/ECT@ECT, Amita Gosalia/LON/ECT@ECT, Mark E Haedicke/HOU/ECT@ECT, Jeff Harbert/ENRON@enronXgate, Greg Hermans/ENRON@enronXgate, Kimberly Hi